In [4]:
# Celula 1 - importuri
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import email
import os
import traceback
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier

# Celula 2 - definirea interfeței Mail Analyser
def open_mail_analyser():
    df = pd.read_csv('../Data/spam.csv', encoding='latin-1')
    df = df[['v1', 'v2']]
    df.columns = ['label', 'text']
    df['label'] = df['label'].map({'spam': 1, 'ham': 0})

    # Împărțirea datelor și antrenarea modelelor (ca în exemplul tău)
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)
    vectorizer = CountVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    models = {
        'Naive Bayes': MultinomialNB(),
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'SVC': SVC(probability=True),
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'Bagging Classifier': BaggingClassifier(),
        'Extra Trees Classifier': ExtraTreesClassifier(),
        'Gradient Boosting': GradientBoostingClassifier(),
        'AdaBoost': AdaBoostClassifier()
    }
    for model in models.values():
        model.fit(X_train_vec, y_train)

    def classify_email(email_text):
        email_vec = vectorizer.transform([email_text])
        results = {}
        for model_name, model in models.items():
            prediction = model.predict(email_vec)[0]
            proba = model.predict_proba(email_vec)[0]
            spam_proba = proba[1] * 100
            ham_proba = proba[0] * 100
            results[model_name] = (spam_proba, ham_proba)
        return results

    mail_analyser_window = tk.Tk()
    mail_analyser_window.title("Mail Analyser")

    def on_classify():
        email_text = email_entry.get("1.0", tk.END).strip()
        if not email_text:
            messagebox.showwarning("Atenție", "Introduceți textul email-ului!")
            return
        results = classify_email(email_text)
        result_text.delete("1.0", tk.END)
        for model_name, (spam_proba, ham_proba) in results.items():
            result_text.insert(tk.END, f"{model_name}: ")
            result_text.insert(tk.END, f"Mesaj autentic ({ham_proba:.2f}%)", "green")
            result_text.insert(tk.END, f" vs Spam ({spam_proba:.2f}%)\n", "red")

    def load_email_file():
        file_path = filedialog.askopenfilename(filetypes=[("Email files", "*.eml")])
        if file_path:
            with open(file_path, 'r', encoding='utf-8') as f:
                msg = email.message_from_file(f)
                email_body = ""
                if msg.is_multipart():
                    for part in msg.walk():
                        if part.get_content_type() == "text/plain":
                            email_body = part.get_payload(decode=True).decode()
                            break
                else:
                    email_body = msg.get_payload(decode=True).decode()
                email_entry.delete("1.0", tk.END)
                email_entry.insert(tk.END, email_body)

    title_label = tk.Label(mail_analyser_window, text="Analiză Email Spam/Ham", font=("Arial", 16, "bold"), bg="#f0f4f7", fg="#333")
    title_label.pack(pady=10)
    
    email_label = tk.Label(mail_analyser_window, text="Introduceți textul email-ului:", font=("Arial", 12), bg="#f0f4f7")
    email_label.pack()
    email_entry = tk.Text(mail_analyser_window, height=10, width=50, font=("Arial", 10))
    email_entry.pack(pady=5)
    
    load_button = tk.Button(mail_analyser_window, text="Încarcă .eml", command=load_email_file, bg="#2196F3", fg="white", font=("Arial", 12, "bold"), width=15)
    load_button.pack(pady=5)
    
    classify_button = tk.Button(mail_analyser_window, text="Clasifică", command=on_classify, bg="#4CAF50", fg="white", font=("Arial", 12, "bold"), width=15)
    classify_button.pack(pady=10)
    
    result_text = tk.Text(mail_analyser_window, height=10, width=80, font=("Arial", 10), bg="#f0f4f7", wrap=tk.WORD)
    result_text.tag_configure("green", foreground="green")
    result_text.tag_configure("red", foreground="red")
    result_text.pack(pady=10, padx=20, fill="both")
    
    mail_analyser_window.mainloop()

# Apelăm funcția pentru a porni interfața
open_mail_analyser()
